In [ ]:
from pyspark.sql import SQLContext
#object storage container to read data from
container = "nrsim"
#an arbitrary logical name for the dataframe
tableName = "nrsim"
#start date for query
iDate = "2017-01-10"
#end date for query
fDate = "2017-01-10"
#parent object name e.g. for "ttt/dt=2017-...." it should be "ttt"
objName = "ttt"

sqlContext = SQLContext(sc)
df = sqlContext.read.json("swift://" + container + ".spark/" + objName) 
df.registerTempTable(tableName)

In [ ]:
import datetime
def runQuery(sqlQuery): # returned a list of the requested rows
    return sqlContext.sql(sqlQuery).collect()

def getXY(x_attr,y_attr,sDate,fDate):
    d = runQuery("SELECT " + str(x_attr) + "," + str(y_attr) + " FROM " + str(tableName) + " WHERE dt BETWEEN '" + \
                 str(sDate) + "' AND '" + str(fDate) + "'")
    xx = []
    yy = []
    for row in d:
        xx.append(datetime.datetime.strptime(getattr(row,x_attr).encode('ascii','ignore'),"%Y-%m-%dT%H:%M:%S"))
        yy.append(int(getattr(row,"velocity")))
    return xx,yy

x,y = getXY("timestamp","payload.velocity",iDate,fDate)
assert(len(x) == len(y))
print x,y

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(x,y)
plt.xlabel('time')
plt.ylabel('velocity')
plt.title('Velocity')
# beautify the x-labels
plt.gcf().autofmt_xdate()
plt.show()